In [ ]:
from typing import List, Literal, Optional, Dict
from pydantic import BaseModel, Field

# ============================
# 0) ENUMS & COMMON TYPES
# ============================

IndexName = Literal["VNINDEX", "VN30", "HNXINDEX", "UPCOM"]
Exchange = Literal["HOSE", "HNX", "UPCOM"]
Language = Literal["vi", "en"]
Audience = Literal["retail", "pro"]
LengthPref = Literal["brief", "standard"]
TonePref = Literal["neutral", "positive", "cautious"]
MarketState = Literal["recovering", "weakening", "rising", "sideways"]

# ============================
# 1) META
# ============================

class Meta(BaseModel):
    date: str = Field(...,
        description="Ngày giao dịch (EOD) theo ISO 8601.",
        example="2025-11-06")
    timezone: str = Field("Asia/Ho_Chi_Minh",
        description="Múi giờ dữ liệu.",
        example="Asia/Ho_Chi_Minh")
    language: Language = Field("vi",
        description="Ngôn ngữ xuất bản bản tin.",
        example="vi")
    audience: Audience = Field("retail",
        description="Đối tượng độc giả (ảnh hưởng giọng điệu, thuật ngữ).",
        example="retail")
    length: LengthPref = Field("standard",
        description="Độ dài bài viết.",
        example="standard")
    tone: TonePref = Field("neutral",
        description="Sắc thái giọng điệu tổng thể.",
        example="neutral")
    source_note: Optional[str] = Field(None,
        description="Ghi chú nguồn dữ liệu (nếu muốn in ra cuối bài).",
        example="Nguồn: Fin68 Data Lake, FiinTrade API")

# ============================
# 2) CORE ENTITIES
# ============================

class TickerRef(BaseModel):
    ticker: str = Field(...,
        description="Mã cổ phiếu UPPERCASE.",
        example="HPG")
    exchange: Exchange = Field(...,
        description="Sàn niêm yết.",
        example="HOSE")
    name: Optional[str] = Field(None,
        description="Tên công ty (nếu có).",
        example="Hoa Phat Group")

class SectorRef(BaseModel):
    sector_code: str = Field(...,
        description="Mã ngành (ví dụ ICB Level 4).",
        example="ICB-1753")
    sector_name: str = Field(...,
        description="Tên ngành.",
        example="Thép")

# ============================
# 3) INTRADAY (5-MIN) – INDICES
# ============================

class IndexBar5m(BaseModel):
    ts: str = Field(...,
        description="Timestamp 5 phút (ISO 8601, có offset +07:00).",
        example="2025-11-06T09:05:00+07:00")
    open: float = Field(...,
        description="Mở cửa bar (điểm chỉ số).",
        example=1234.5)
    high: float = Field(...,
        description="Cao nhất bar (điểm chỉ số).",
        example=1236.8)
    low: float = Field(...,
        description="Thấp nhất bar (điểm chỉ số).",
        example=1233.9)
    close: float = Field(...,
        description="Đóng cửa bar (điểm chỉ số).",
        example=1236.2)
    value_bil_vnd: float = Field(...,
        description="Giá trị khớp lệnh trong bar (tỷ VND).",
        example=980.0)
    cum_value_bil_vnd: Optional[float] = Field(None,
        description="Giá trị khớp lệnh lũy kế đến bar (tỷ VND).",
        example=980.0)
    foreign_net_bil_vnd: Optional[float] = Field(None,
        description="Khối ngoại mua/bán ròng trong bar (tỷ VND). Dương = mua ròng.",
        example=-15.0)
    cum_foreign_net_bil_vnd: Optional[float] = Field(None,
        description="Khối ngoại ròng lũy kế đến bar (tỷ VND).",
        example=-30.0)
    active_net_bil_vnd: Optional[float] = Field(None,
        description="Dòng tiền ròng chủ động (mua chủ động - bán chủ động) trong bar (tỷ VND).",
        example=25.0)
    cum_active_net_bil_vnd: Optional[float] = Field(None,
        description="Dòng tiền ròng chủ động lũy kế đến bar (tỷ VND).",
        example=40.0)
    advancers: Optional[int] = Field(None,
        description="Số mã tăng tại thời điểm bar (toàn sàn liên quan).",
        example=280)
    decliners: Optional[int] = Field(None,
        description="Số mã giảm tại thời điểm bar.",
        example=140)
    unchanged: Optional[int] = Field(None,
        description="Số mã đứng giá tại thời điểm bar.",
        example=60)

class IndexIntradaySeries(BaseModel):
    index: IndexName = Field(...,
        description="Tên chỉ số.",
        example="VNINDEX")
    bars: List[IndexBar5m] = Field(...,
        description="Danh sách bar 5 phút theo thời gian tăng dần.",
        example=[])

# ============================
# 4) INTRADAY (5-MIN) – GENERIC FLOWS (SECTOR & TICKER)
# ============================

class FlowBar5m(BaseModel):
    ts: str = Field(...,
        description="Timestamp 5 phút (ISO 8601 +07:00).",
        example="2025-11-06T10:35:00+07:00")
    value_bil_vnd: Optional[float] = Field(None,
        description="GT khớp trong bar (tỷ VND).",
        example=120.0)
    foreign_net_bil_vnd: Optional[float] = Field(None,
        description="Khối ngoại ròng trong bar (tỷ VND).",
        example=-5.0)
    active_net_bil_vnd: Optional[float] = Field(None,
        description="Dòng tiền ròng chủ động trong bar (tỷ VND).",
        example=8.0)

class SectorFlowIntraday(BaseModel):
    sector: SectorRef = Field(...,
        description="Tham chiếu ngành.")
    bars: List[FlowBar5m] = Field(...,
        description="Chuỗi 5 phút cho ngành.")

class TickerFlowIntraday(BaseModel):
    ticker: TickerRef = Field(...,
        description="Tham chiếu cổ phiếu.")
    bars: List[FlowBar5m] = Field(...,
        description="Chuỗi 5 phút cho mã.")

class IntradayFlows(BaseModel):
    indices: Optional[List[IndexIntradaySeries]] = Field(None,
        description="Chuỗi intraday 5m cho 4 chỉ số.")
    sectors: Optional[List[SectorFlowIntraday]] = Field(None,
        description="Chuỗi intraday 5m cho một số ngành nổi bật (top theo GT/flow).")
    tickers: Optional[List[TickerFlowIntraday]] = Field(None,
        description="Chuỗi intraday 5m cho một số mã nổi bật (top theo GT/flow).")

# ============================
# 5) EOD SNAPSHOTS – INDICES, SECTORS, TICKERS
# ============================

class IndexEOD(BaseModel):
    index: IndexName = Field(..., description="Chỉ số.", example="VNINDEX")
    open: float = Field(..., description="Giá mở cửa ngày (điểm).", example=1234.5)
    high: float = Field(..., description="Đỉnh ngày (điểm).", example=1242.1)
    low: float = Field(..., description="Đáy ngày (điểm).", example=1228.0)
    close: float = Field(..., description="Đóng cửa ngày (điểm).", example=1238.4)
    change_abs: float = Field(..., description="Thay đổi tuyệt đối (điểm).", example=6.3)
    change_pct: float = Field(..., description="Thay đổi % so với phiên trước.", example=0.51)
    matched_value_bil_vnd: float = Field(...,
        description="Tổng GT khớp EOD (tỷ VND).", example=17800.0)
    foreign_net_bil_vnd: float = Field(...,
        description="Khối ngoại ròng EOD (tỷ VND). Dương = mua ròng.", example=-350.0)
    active_net_bil_vnd: float = Field(...,
        description="Dòng tiền ròng chủ động EOD (tỷ VND).", example=220.0)
    advancers: Optional[int] = Field(None, description="Tổng số mã tăng trong ngày.", example=290)
    decliners: Optional[int] = Field(None, description="Tổng số mã giảm trong ngày.", example=210)
    unchanged: Optional[int] = Field(None, description="Số mã đứng giá.", example=60)
    limit_up: Optional[int] = Field(None, description="Số mã tăng trần.", example=6)
    limit_down: Optional[int] = Field(None, description="Số mã giảm sàn.", example=2)

class SectorEOD(BaseModel):
    sector: SectorRef = Field(..., description="Ngành.")
    open: float = Field(..., description="Open ngành (index ngành hoặc composite).", example=1012.3)
    high: float = Field(..., description="High ngành.", example=1025.0)
    low: float = Field(..., description="Low ngành.", example=1005.8)
    close: float = Field(..., description="Close ngành.", example=1019.6)
    change_pct: float = Field(..., description="% thay đổi ngày của ngành.", example=0.65)
    matched_value_bil_vnd: float = Field(..., description="GT khớp EOD của ngành (tỷ VND).", example=2200.0)
    foreign_net_bil_vnd: float = Field(..., description="Khối ngoại ròng EOD của ngành (tỷ VND).", example=-80.0)
    active_net_bil_vnd: float = Field(..., description="Dòng tiền ròng chủ động EOD ngành (tỷ VND).", example=55.0)
    contrib_to_vnindex_pts: Optional[float] = Field(None,
        description="Đóng góp điểm của ngành vào VNINDEX (nếu có) – dương/âm.",
        example=1.8)

class TickerEOD(BaseModel):
    ticker: TickerRef = Field(..., description="Mã cổ phiếu.")
    close: float = Field(..., description="Giá đóng cửa.", example=28.75)
    change_pct: float = Field(..., description="% thay đổi so với EOD trước.", example=2.35)
    matched_value_bil_vnd: float = Field(..., description="GT khớp EOD (tỷ VND).", example=950.0)
    foreign_net_bil_vnd: float = Field(..., description="Khối ngoại ròng EOD (tỷ VND).", example=-60.0)
    active_net_bil_vnd: float = Field(..., description="Dòng tiền ròng chủ động EOD (tỷ VND).", example=35.0)
    sector_code: Optional[str] = Field(None, description="Ngành của mã (nếu cần).", example="ICB-1753")
    contrib_to_vnindex_pts: Optional[float] = Field(None,
        description="Đóng góp điểm của mã vào VNINDEX (nếu thuộc rổ tính).",
        example=0.35)

# ============================
# 6) CONTRIBUTIONS (VNINDEX)
# ============================

class SectorContribution(BaseModel):
    sector: SectorRef = Field(..., description="Ngành.")
    contrib_pts: float = Field(...,
        description="Đóng góp điểm vào VNINDEX (dương/âm).",
        example=0.95)
    change_pct: Optional[float] = Field(None,
        description="% thay đổi chỉ số ngành trong ngày.",
        example=1.10)

class TickerContribution(BaseModel):
    ticker: TickerRef = Field(..., description="Mã cổ phiếu.")
    contrib_pts: float = Field(...,
        description="Đóng góp điểm vào VNINDEX (dương/âm).",
        example=0.22)

class ContributionsVNIndex(BaseModel):
    sectors: List[SectorContribution] = Field(...,
        description="Danh sách đóng góp theo ngành, đã sắp xếp theo |contrib_pts| giảm dần.")
    tickers: List[TickerContribution] = Field(...,
        description="Danh sách đóng góp theo mã, đã sắp xếp theo |contrib_pts| giảm dần.")

# ============================
# 7) TOP LISTS (COMMON CRITERIA)
# ============================

class TickerMove(BaseModel):
    ticker: TickerRef = Field(..., description="Mã cổ phiếu.")
    change_pct: float = Field(..., description="% thay đổi ngày.", example=3.2)
    matched_value_bil_vnd: float = Field(..., description="GT khớp EOD (tỷ VND).", example=780.0)
    foreign_net_bil_vnd: float = Field(..., description="Khối ngoại ròng (tỷ VND).", example=45.0)
    active_net_bil_vnd: float = Field(..., description="Dòng tiền ròng chủ động (tỷ VND).", example=60.0)

class TopLists(BaseModel):
    top_gainers: List[TickerMove] = Field(...,
        description="Top tăng % (ưu tiên vốn hoá lớn nếu muốn).")
    top_losers: List[TickerMove] = Field(...,
        description="Top giảm %.")
    top_liquidity: List[TickerMove] = Field(...,
        description="Top GT khớp EOD cao nhất.")
    top_foreign_buy: List[TickerMove] = Field(...,
        description="Top khối ngoại mua ròng.")
    top_foreign_sell: List[TickerMove] = Field(...,
        description="Top khối ngoại bán ròng.")
    top_active_buy: List[TickerMove] = Field(...,
        description="Top dòng tiền chủ động mua ròng.")
    top_active_sell: List[TickerMove] = Field(...,
        description="Top dòng tiền chủ động bán ròng.")
    top_contributors_up: List[TickerContribution] = Field(...,
        description="Top mã đóng góp dương vào VNINDEX (điểm).")
    top_contributors_down: List[TickerContribution] = Field(...,
        description="Top mã đóng góp âm vào VNINDEX (điểm).")
    top_unusual_volume: Optional[List[TickerRef]] = Field(None,
        description="Top khối lượng bất thường (ví dụ zscore vol ≥ 2).",
        example=[{"ticker":"HPG","exchange":"HOSE","name":"Hoa Phat Group"}])
    top_52w_high_breakout: Optional[List[TickerRef]] = Field(None,
        description="Danh sách mã vượt đỉnh 52W.",
        example=[{"ticker":"FPT","exchange":"HOSE","name":"FPT Corp"}])
    top_52w_low_breakdown: Optional[List[TickerRef]] = Field(None,
        description="Danh sách mã thủng đáy 52W.",
        example=[{"ticker":"VIC","exchange":"HOSE","name":"Vingroup"}])

# ============================
# 8) MARKET STATE FEATURES (for LLM inference)
# ============================

class MarketStateFeatures(BaseModel):
    index: IndexName = Field(..., description="Chỉ số cần đánh giá.", example="VNINDEX")

    # Return & volatility signatures
    day_return_pct: float = Field(...,
        description="% thay đổi EOD của chỉ số.",
        example=0.51)
    intraday_range_pts: float = Field(...,
        description="Range ngày = high - low (điểm).",
        example=14.1)
    intraday_vol_proxy_pts: Optional[float] = Field(None,
        description="Proxy biến động nội ngày (ví dụ std 5m close hoặc ATR-5m).",
        example=3.8)

    # Breadth (early vs late) – dùng để suy ra phục hồi/suy yếu
    breadth_open_adv: Optional[int] = Field(None,
        description="Số mã tăng giai đoạn đầu phiên (ví dụ 09:00–09:30).",
        example=290)
    breadth_open_dec: Optional[int] = Field(None, description="Số mã giảm đầu phiên.", example=160)
    breadth_close_adv: Optional[int] = Field(None, description="Số mã tăng cuối phiên.", example=275)
    breadth_close_dec: Optional[int] = Field(None, description="Số mã giảm cuối phiên.", example=210)

    # VWAP & reversal
    close_vs_vwap_pts: Optional[float] = Field(None,
        description="Độ lệch Close so với VWAP (điểm). Dương → đóng cửa trên VWAP.",
        example=1.1)
    rebound_from_low_pts: Optional[float] = Field(None,
        description="Mức hồi từ đáy nội ngày đến Close (điểm).",
        example=5.4)

    # Flows
    foreign_net_bil_vnd: float = Field(...,
        description="Khối ngoại ròng cả ngày (tỷ VND).",
        example=-350.0)
    active_net_bil_vnd: float = Field(...,
        description="Dòng tiền ròng chủ động cả ngày (tỷ VND).",
        example=220.0)
    active_flow_reversal: Optional[bool] = Field(None,
        description="True nếu dòng tiền chủ động từ âm → dương trong ngày.",
        example=True)

    # Contributions concentration (để biết dẫn dắt tập trung hay lan toả)
    top3_sector_contrib_share: Optional[float] = Field(None,
        description="Tỷ trọng đóng góp của 3 ngành lớn nhất trên tổng |đóng góp| (0–1).",
        example=0.62)
    top5_ticker_contrib_share: Optional[float] = Field(None,
        description="Tỷ trọng đóng góp của 5 mã lớn nhất trên tổng |đóng góp| (0–1).",
        example=0.55)

    # Trend context (nếu có sẵn)
    distance_ma20_pct: Optional[float] = Field(None,
        description="Khoảng cách Close so với MA20 (%).",
        example=1.8)
    distance_ma50_pct: Optional[float] = Field(None,
        description="Khoảng cách Close so với MA50 (%).",
        example=0.9)

    # Optional hint (nếu backend muốn đề xuất trước)
    state_hint: Optional[MarketState] = Field(None,
        description="Gợi ý trạng thái nếu đã tính trước (LLM có thể tham khảo, không bắt buộc).",
        example="recovering")

# ============================
# 9) LIQUIDITY & FOREIGN SUMMARY BY EXCHANGE
# ============================

class LiquidityByExchange(BaseModel):
    exchange: Exchange = Field(..., description="Sàn.", example="HOSE")
    matched_value_bil_vnd: float = Field(...,
        description="GT khớp toàn sàn (tỷ VND).", example=17800.0)
    vs_prev_pct: Optional[float] = Field(None,
        description="% so với phiên trước.", example=-3.2)

class ForeignFlowByExchange(BaseModel):
    exchange: Literal["ALL", "HOSE", "HNX", "UPCOM"] = Field(...,
        description="Phạm vi giao dịch khối ngoại.",
        example="ALL")
    net_value_bil_vnd: float = Field(...,
        description="Giá trị ròng (tỷ VND). Dương = mua ròng.",
        example=-350.0)
    top_buy: List[str] = Field(default_factory=list,
        description="Danh sách mã khối ngoại mua ròng nhiều.",
        example=["VCB","VHM"])
    top_sell: List[str] = Field(default_factory=list,
        description="Danh sách mã khối ngoại bán ròng nhiều.",
        example=["HPG","SSI","STB"])

# ============================
# 10) MASTER INPUT
# ============================

class MarketCloseReportInput(BaseModel):
    meta: Meta = Field(..., description="Thông tin chung & cấu hình xuất bản.")
    indices_eod: List[IndexEOD] = Field(...,
        description="EOD snapshots cho 4 chỉ số.")
    sectors_eod: Optional[List[SectorEOD]] = Field(None,
        description="EOD OHLCV + flows cho các ngành (ít nhất top ngành).")
    contributions_vnindex: Optional[ContributionsVNIndex] = Field(None,
        description="Đóng góp điểm theo ngành và mã vào VNINDEX.")
    intraday: Optional[IntradayFlows] = Field(None,
        description="Chuỗi intraday 5m cho chỉ số/ngành/mã (chọn lọc để tránh phình token).")
    top_lists: TopLists = Field(...,
        description="Các bảng xếp hạng phổ biến dùng trong bản tin.")
    liquidity_summary: List[LiquidityByExchange] = Field(...,
        description="Tổng GT khớp theo sàn.")
    foreign_summary: List[ForeignFlowByExchange] = Field(...,
        description="Tổng hợp giao dịch khối ngoại theo sàn/ALL.")
    state_features: List[MarketStateFeatures] = Field(...,
        description="Bộ feature để LLM suy luận trạng thái thị trường cho từng chỉ số.")



In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import json
from function.market.market_summary import build_market_summary
prompt = ChatPromptTemplate.from_template("""
Bạn là chuyên gia thị trường chứng khoán Việt Nam. Dưới đây là dữ liệu thống kê hôm nay đã TÍNH SẴN (KHÔNG tính lại); `change_percent_vs_prev_day` và `change_points_vs_prev_day` là biến động `index_value` so với ngày hôm trước:

{data}

YÊU CẦU: Viết phần đầu tiên của Flash Note: "Thanh khoản & Biến động chỉ số" (120–180 từ), tiếng Việt, súc tích, chuyên nghiệp.
BẮT BUỘC gồm:
1) Thanh khoản thị trường so với trung bình 20 phiên: dùng field pct_vs_avg_20d & z_score_liquidity  của từng sàn để mô tả mức độ (ví dụ: rất thấp/ thấp/ trung tính/ cao/ rất cao) không nêu chỉ số thanh khoản z_score_liquidity trong phần viết chỉ dùng để phân tích đưa ra nhận định.
2) Tóm tắt biến động chỉ số (mỗi sàn 1 câu): dùng change_points_vs_prev_day & change_percent_vs_prev_day.
3) Không suy đoán, không tính lại số, không viết công thức.
4) z_score_liquidity, z_bucket và σ bạn ngầm sử dụng không nêu các thông số này ra
Giọng điệu: trung lập, định lượng, trực diện.
""")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.35)
chain = prompt | llm | StrOutputParser()

market_summary = json.dumps(build_market_summary(), ensure_ascii=False, indent=2) # market_summary là dict bạn đã có

flash_note_liquidity = chain.invoke({"data": json.dumps(market_summary, ensure_ascii=False, indent=2)})
print(flash_note_liquidity)


📦 Closing DB session
**Thanh khoản & Biến động chỉ số**

Trong phiên giao dịch hôm nay, thanh khoản trên các sàn chứng khoán có sự phân hóa rõ rệt. Tại sàn HOSE, thanh khoản đạt mức trung tính so với trung bình 20 phiên, cho thấy hoạt động giao dịch ổn định. Trong khi đó, sàn HNX ghi nhận thanh khoản ở mức thấp, cho thấy sự thận trọng của nhà đầu tư. Sàn UPCoM có thanh khoản cũng ở mức trung tính, phản ánh sự cân bằng trong giao dịch.

Về biến động chỉ số, VN-Index tăng 5.5 điểm, tương đương với mức tăng 0.33% so với ngày trước đó, cho thấy xu hướng tích cực. VN30 cũng ghi nhận sự tăng trưởng nhẹ với 4.53 điểm, tương ứng 0.24%. Ngược lại, HNX giảm 1.33 điểm, tương đương -0.49%, và UPCoM giảm 0.66 điểm, tương ứng -0.55%, cho thấy áp lực giảm giá tại hai sàn này.


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import json



from function.market.breadth_index import build_market_breadth
# ===== 1) Meta context: định nghĩa thang đo & cách diễn đạt =====
meta_context = """
ĐỊNH NGHĨA THANG ĐỘ RỘNG (breadth_bucket):
- bullish: lan tỏa tích cực, phần lớn cổ phiếu tăng.
- slightly_bullish: nghiêng tăng, sắc xanh nhỉnh hơn.
- neutral: cân bằng, xanh/đỏ tương đương.
- slightly_bearish: nghiêng giảm, sắc đỏ nhỉnh hơn.
- bearish: lan tỏa tiêu cực, phần lớn cổ phiếu giảm.

QUY ƯỚC HIỂN THỊ SỐ:
- Mỗi sàn hiển thị số lượng theo dạng (A/D/U = advancers/decliners/unchanged).
- Có thể dùng 1–2 tính từ ngắn gọn (ví dụ: “lan tỏa tiêu cực”, “nghiêng giảm”).
- Giọng văn: trung lập, định lượng, chuyên nghiệp; không suy đoán nguyên nhân.
"""

# ===== 2) Prompt: yêu cầu LLM dùng bucket để diễn đạt + show số A/D/U =====
prompt = ChatPromptTemplate.from_template("""
Bạn là chuyên gia phân tích thị trường chứng khoán Việt Nam.

{meta_context}

DỮ LIỆU HÔM NAY (đã tính sẵn, KHÔNG tính lại):
{data}

YÊU CẦU:
1) Viết mục “Độ rộng thị trường” 80–120 từ, tiếng Việt, trung lập, định lượng.
2) Mở đầu 1 câu nêu bức tranh chung toàn thị trường (dựa trên các breadth_bucket).
3) Sau đó tóm tắt từng sàn (VNINDEX, HNX, UPCOM):
   - Diễn đạt theo breadth_bucket.description.
   - Hiển thị số lượng mã tăng/giảm/không đổi.
4) Không nêu công thức. Không phỏng đoán động cơ.

Lưu ý:
- Nếu sàn nào vắng dữ liệu thì bỏ qua sàn đó.
- Giữ câu văn ngắn, trực diện.
Trả lời bằng một đoạn văn duy nhất, không dùng bullet.
""")

# ===== 3) LLM + chain =====
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.35)
chain = prompt | llm | StrOutputParser()


breadth_data=build_market_breadth()

# ===== 5) Invoke =====
data_json = json.dumps(breadth_data, ensure_ascii=False, indent=2)
breadth_market = chain.invoke({"meta_context": meta_context, "data": data_json})
print(breadth_market)


📦 Closing DB session
Độ rộng thị trường hôm nay cho thấy sự giằng co, với các chỉ số ở trạng thái cân bằng. Trên sàn VNINDEX, số lượng cổ phiếu tăng là 162, trong khi số mã giảm là 166, cho thấy độ rộng cân bằng. Tương tự, sàn HNX ghi nhận 57 mã tăng và 73 mã giảm, cũng phản ánh tình trạng cân bằng. Trong khi đó, sàn UPCOM có 143 mã tăng và 108 mã giảm, nhưng vẫn giữ được trạng thái cân bằng. Tổng thể, thị trường hiện đang trong giai đoạn giằng co với sự phân hóa giữa các mã cổ phiếu.


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import json
from function.market.foreign_flow_summary import build_foreign_flow_summary 
foreign_breath=build_foreign_flow_summary()
foreign_breath
meta_context = meta_context = """
QUY ƯỚC:
- Dấu: âm = bán ròng, dương = mua ròng. Đơn vị giá trị: tỷ VND.
- Không tính toán lại từ dữ liệu; chỉ diễn đạt theo thông tin đã cung cấp.
- 'magnitude_bucket_*' và 'breadth_bucket' là mức độ định tính 5 bậc (từ strongly_negative → strongly_positive / bearish → bullish).
- Xử lý mảng rỗng: nếu top_buy_sectors['items'] hoặc top_sell_sectors['items'] không có phần tử, ghi rõ không ghi nhận nhóm tương ứng.
- Khi nêu cổ phiếu trong mỗi ngành, chỉ nêu tối đa 5 mã **tiêu biểu** kèm giá trị theo thứ tự đã cung cấp.
"""


prompt = ChatPromptTemplate.from_template("""
Bạn là chuyên gia chứng khoán Việt Nam.

{meta_context}

DỮ LIỆU KHỐI NGOẠI (đã tính sẵn, KHÔNG tính lại):
{data}

YÊU CẦU , giọng trung lập – định lượng – trực diện:
1) Mở đầu: nêu trạng thái dòng tiền ròng của khối ngoại trên toàn thị trường theo 'dir_total'.
2) Ngành tiêu biểu:
   - Ưu tiên phía nêu **áp đảo** (nếu 'dir_total' là net_sell thì ưu tiên nhóm bán) trước sau đó đến nhóm mua, liệt kê các ngành và giá trị.
   - Mỗi ngành nêu các mã và giá trị từ 'top_stocks['items'] '.
3) Viết không công thức, không suy đoán nguyên nhân.
4) Không bullet viết theo lối hành văn 

Ghi nhớ:
- Không trích dẫn hoặc suy luận bất kỳ con số nào nếu không được yêu cầu.
- Tên ngành lấy từ 'sector_name'; mã cổ phiếu lấy từ 'symbol'.
""")

# dùng:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.35)
chain = prompt | llm | StrOutputParser()
foreign_breath_content = chain.invoke({"meta_context": meta_context, "data": json.dumps(foreign_breath, ensure_ascii=False, indent=2)})
print(foreign_breath_content)



📦 Closing DB session
Tính đến ngày 18 tháng 11 năm 2025, dòng tiền ròng của khối ngoại trên toàn thị trường là bán ròng.

Trong nhóm ngành bán, ngành Môi giới chứng khoán dẫn đầu với giá trị bán ròng là -350.3 tỷ VND. Các mã cổ phiếu tiêu biểu trong ngành này bao gồm VIX với -151.3 tỷ VND, VCI với -131.4 tỷ VND, VND với -59.7 tỷ VND và HCM với -47.4 tỷ VND. Ngành Kho bãi, hậu cần và bảo dưỡng theo sau với giá trị bán ròng -100.4 tỷ VND, trong đó có ACV với -48.0 tỷ VND và VSC với -32.4 tỷ VND. Ngành Thực phẩm cũng ghi nhận bán ròng -80.2 tỷ VND, với MSN đạt -64.3 tỷ VND và MCH đạt -40.7 tỷ VND.

Về phía mua, ngành Thép và sản phẩm thép dẫn đầu với giá trị mua ròng 374.2 tỷ VND, trong đó HPG đạt 377.6 tỷ VND. Ngành Phần mềm ghi nhận mua ròng 107.1 tỷ VND với mã FPT đạt 107.1 tỷ VND. Ngành Thiết bị và Dịch vụ Dầu khí có giá trị mua ròng 93.9 tỷ VND, với PVS đạt 56.2 tỷ VND và PVD đạt 37.6 tỷ VND.


In [4]:
# -*- coding: utf-8 -*-
from __future__ import annotations

from typing import List, Optional, Dict, Any
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser,StrOutputParser
from IPython.display import display, Markdown

# ===== 0) SCHEMA KẾT QUẢ: ép LLM trả về có cấu trúc =====

class Section(BaseModel):
    """Một mục văn bản đã biên soạn, 80–180 từ/mục, tiếng Việt, trung lập – định lượng – trực diện."""
    title: str = Field(..., description="Tiêu đề mục")
    text: str = Field(..., description="Nội dung mục (đoạn văn, không bullet)")

class MarketOverview(BaseModel):
    """Tổng quan thị trường – bản tin hợp nhất để xuất bản."""
    date: str = Field(..., description="Ngày dữ liệu, YYYY-MM-DD (lấy từ input)")
    headline: str = Field(..., description="Một câu tiêu đề súc tích về trạng thái thị trường hôm nay")
    summary_bullets: List[str] = Field(..., description="3–5 ý chính ngắn gọn")
    sections: Dict[str, Section] = Field(
        ...,
        description="Các mục đã biên soạn: liquidity_and_indices, market_breadth, foreign_flow, notable_sectors (tuỳ), risk_watch (tuỳ)"
    )
    final_text: str = Field(
        ...,
        description="Toàn bộ bản tin Market Overview đã ghép liền mạch (300–450 từ), tiếng Việt, không bullet"
    )

# ===== 1) META CONTEXT – quy ước dùng chung cho mọi mục =====

META_CONTEXT = """
QUY ƯỚC CHUNG (KHÔNG tính lại số):
- Chỉ diễn đạt theo số liệu đã cung cấp không suy đoán nguyên nhân, không ngoại suy.
- VN30 là: "chỉ số VN30" không phải là sàn. HNINDEX thuộc sàn HNX. VNINDEX là chỉ số đại diện của sàn HOSE
- không sử dụng mẫu "*, và" hãy sử dụng "* và *" (với * là chuỗi ký tự) không để dấu , đứng trước từ "và" 
- Dấu: âm = bán ròng, dương = mua ròng; đơn vị giá trị: tỷ VND.
- Không nêu công thức, không đưa tham số nội bộ (z-score, σ…).
- Giọng văn: trung lập – định lượng – trực diện; câu ngắn, rõ nghĩa.
- Thống nhất cách gọi sàn: VNINDEX (HOSE), HNX, UPCOM.

GỢI Ý DIỄN ĐẠT:
- Thanh khoản: mô tả mức độ so với trung bình 20 phiên bằng các cụm "rất thấp / thấp / trung tính / cao / rất cao" dựa trên pct_vs_avg_20d và z_score_liquidity đã bucket sẵn (nếu có).
- Biến động chỉ số: mỗi sàn 1 câu, dùng change_points_vs_prev_day & change_percent_vs_prev_day.
- Breadth: dùng breadth_bucket và A/D/U; diễn đạt theo: bullish / slightly_bullish / neutral / slightly_bearish / bearish.
- Khối ngoại: tóm lược trạng thái toàn thị trường (dir_total) rồi nêu các ngành tiêu biểu (ưu tiên phía áp đảo) rồi dến phía còn lại, liệt kê mã tiêu biểu theo thứ tự đã cho.
- Xử lý thiếu dữ liệu: sàn/ngành nào không có dữ liệu thì bỏ qua, không suy đoán bù.
"""

# ===== 2) PROMPT HỢP NHẤT =====

prompt_template = ChatPromptTemplate.from_template("""
Bạn là chuyên gia thị trường chứng khoán Việt Nam. Hãy tạo bản tin Market Overview đá có dữ liệu đầu vào với các nội dung dưới đây.

{meta_context}

DỮ LIỆU ĐẦU VÀO (đã tính sẵn, KHÔNG tính lại):
- market_summary (liquidity & index move):
{market_summary}

- market_breadth (breadth A/D/U + breadth_bucket):
{market_breadth}

- foreign_flow (khối ngoại – dòng tiền ròng & top ngành/cổ phiếu):
{foreign_flow}

YÊU CẦU XUẤT RA:

- Nội dung trả lời ghép mạch lạc các mục trên theo lối hành văn, không lặp từ. Markdown phù hợp đối với nền tảng telegram các bullet thay bằng icon phù hợp của telegram nhé
- Style có icon để cho bài viết sống động hơn nhé!. Lồng ghép icon hợp lý nha.
- Có phần tổng kết các thông tin 
                                                                                                      
LƯU Ý:
- Không dùng số mới hoặc suy đoán. Chỉ dùng trường đã cấp.
- Không chèn ký hiệu σ, z-score, tên bucket nội bộ.
- Giữ văn phong súc tích, định lượng, trung lập.
- Nếu thiếu dữ liệu của sàn/ngành nào, hãy lược bỏ phần đó mạch lạc.

""")

# ===== 3) KHỞI TẠO LLM + PARSER =====

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
# parser = PydanticOutputParser(pydantic_object=MarketOverview)
structured_chain = prompt_template | llm | StrOutputParser()

# ===== 5) VÍ DỤ SỬ DỤNG (khớp với code bạn đã có) =====
result = structured_chain.invoke({
    "meta_context": META_CONTEXT,
    "market_summary": json.dumps(flash_note_liquidity, ensure_ascii=False, indent=2),
    "market_breadth": json.dumps(breadth_market, ensure_ascii=False, indent=2),
    "foreign_flow": json.dumps(foreign_breath_content, ensure_ascii=False, indent=2),
})

display(Markdown(result))


📈 **Bản tin Thị trường Chứng khoán Việt Nam**

---

**Thanh khoản & Biến động chỉ số**  

Trong phiên giao dịch hôm nay, thanh khoản trên các sàn chứng khoán có sự phân hóa rõ rệt. Tại sàn HOSE, thanh khoản đạt mức trung tính so với trung bình 20 phiên, cho thấy hoạt động giao dịch ổn định. Sàn HNX ghi nhận thanh khoản ở mức thấp, cho thấy sự thận trọng của nhà đầu tư. Sàn UPCoM có thanh khoản cũng ở mức trung tính, phản ánh sự cân bằng trong giao dịch.

Về biến động chỉ số, VNINDEX tăng 5.5 điểm, tương đương với mức tăng 0.33% so với ngày trước đó, cho thấy xu hướng tích cực. VN30 cũng ghi nhận sự tăng trưởng nhẹ với 4.53 điểm, tương ứng 0.24%. Ngược lại, HNX giảm 1.33 điểm, tương đương -0.49%, và UPCoM giảm 0.66 điểm, tương ứng -0.55%, cho thấy áp lực giảm giá tại hai sàn này.

---

**Độ rộng thị trường**  

Độ rộng thị trường hôm nay cho thấy sự giằng co, với các chỉ số ở trạng thái cân bằng. Trên sàn VNINDEX, số lượng cổ phiếu tăng là 162, trong khi số mã giảm là 166, cho thấy độ rộng cân bằng. Tương tự, sàn HNX ghi nhận 57 mã tăng và 73 mã giảm, cũng phản ánh tình trạng cân bằng. Sàn UPCoM có 143 mã tăng và 108 mã giảm, nhưng vẫn giữ được trạng thái cân bằng. Tổng thể, thị trường hiện đang trong giai đoạn giằng co với sự phân hóa giữa các mã cổ phiếu.

---

**Dòng tiền khối ngoại**  

Tính đến ngày 18 tháng 11 năm 2025, dòng tiền ròng của khối ngoại trên toàn thị trường là bán ròng.  

Ngành Môi giới chứng khoán dẫn đầu với giá trị bán ròng -350.3 tỷ VND. Các mã cổ phiếu tiêu biểu trong ngành này bao gồm VIX với -151.3 tỷ VND, VCI với -131.4 tỷ VND, VND với -59.7 tỷ VND và HCM với -47.4 tỷ VND. Ngành Kho bãi, hậu cần và bảo dưỡng theo sau với giá trị bán ròng -100.4 tỷ VND, trong đó có ACV với -48.0 tỷ VND và VSC với -32.4 tỷ VND. Ngành Thực phẩm cũng ghi nhận bán ròng -80.2 tỷ VND, với MSN đạt -64.3 tỷ VND và MCH đạt -40.7 tỷ VND.

Về phía mua, ngành Thép và sản phẩm thép dẫn đầu với giá trị mua ròng 374.2 tỷ VND, trong đó HPG đạt 377.6 tỷ VND. Ngành Phần mềm ghi nhận mua ròng 107.1 tỷ VND với mã FPT đạt 107.1 tỷ VND. Ngành Thiết bị và Dịch vụ Dầu khí có giá trị mua ròng 93.9 tỷ VND, với PVS đạt 56.2 tỷ VND và PVD đạt 37.6 tỷ VND.

---

🔍 **Tổng kết**  
Thị trường chứng khoán hôm nay ghi nhận sự phân hóa rõ rệt về thanh khoản và biến động chỉ số. VNINDEX và VN30 tăng nhẹ, trong khi HNX và UPCoM giảm. Độ rộng thị trường cho thấy sự giằng co giữa các mã cổ phiếu. Dòng tiền khối ngoại chủ yếu bán ròng, tập trung vào ngành Môi giới chứng khoán, trong khi ngành Thép và sản phẩm thép ghi nhận mua ròng mạnh.  

--- 

Chúc các nhà đầu tư giao dịch hiệu quả! 📊

In [6]:
print(result)

📈 **Bản tin Thị trường Chứng khoán Việt Nam**

---

**Thanh khoản & Biến động chỉ số**  

Trong phiên giao dịch hôm nay, thanh khoản trên các sàn chứng khoán có sự phân hóa rõ rệt. Tại sàn HOSE, thanh khoản đạt mức trung tính so với trung bình 20 phiên, cho thấy hoạt động giao dịch ổn định. Sàn HNX ghi nhận thanh khoản ở mức thấp, cho thấy sự thận trọng của nhà đầu tư. Sàn UPCoM có thanh khoản cũng ở mức trung tính, phản ánh sự cân bằng trong giao dịch.

Về biến động chỉ số, VNINDEX tăng 5.5 điểm, tương đương với mức tăng 0.33% so với ngày trước đó, cho thấy xu hướng tích cực. VN30 cũng ghi nhận sự tăng trưởng nhẹ với 4.53 điểm, tương ứng 0.24%. Ngược lại, HNX giảm 1.33 điểm, tương đương -0.49%, và UPCoM giảm 0.66 điểm, tương ứng -0.55%, cho thấy áp lực giảm giá tại hai sàn này.

---

**Độ rộng thị trường**  

Độ rộng thị trường hôm nay cho thấy sự giằng co, với các chỉ số ở trạng thái cân bằng. Trên sàn VNINDEX, số lượng cổ phiếu tăng là 162, trong khi số mã giảm là 166, cho thấy độ r

In [ ]:


# 🔎 **BÁO CÁO PHÂN TÍCH – PVT (Tổng CTCP Vận tải Dầu khí)**

**Khuyến nghị: KHẢ QUAN • Giá mục tiêu 22.000 VNĐ/cp • Upside: 18.6%**

---

## **1. Kết luận đầu tư (Investment Thesis)**

PVT là doanh nghiệp vận tải dầu khí hàng đầu Việt Nam với nền tảng tài chính vững và đội tàu lớn nhất thị trường nội địa. Doanh thu 9T2025 tăng trưởng tích cực nhờ mở rộng mảng vận tải và thương mại, song lợi nhuận Q3/2025 giảm do biên lợi nhuận thu hẹp và chi phí tài chính tăng.

Trong trung – dài hạn, động lực tăng trưởng đến từ chiến lược mở rộng đội tàu lên 80–100 chiếc và đa dạng hóa mảng kinh doanh sang hóa chất, hàng rời và tàu thành phẩm. Giá cho thuê tàu định hạn duy trì ổn định giúp tạo mặt bằng doanh thu bền vững, trong khi chi phí có thể giảm từ 2026 nhờ tối ưu vận hành.

Với tiềm năng tăng giá 18.6%, báo cáo đưa ra **khuyến nghị KHẢ QUAN**.

---

## **2. Khuyến nghị và giá mục tiêu**

### **2.1. Khuyến nghị KHẢ QUAN**

* **Giá mục tiêu:** 22.000 VNĐ/cp
* **Upside 12 tháng:** **18.6%**
* **Đánh giá:** Triển vọng tăng trưởng dài hạn hấp dẫn nhờ mở rộng đội tàu và đa dạng hóa hoạt động kinh doanh, dù chịu áp lực lợi nhuận trong ngắn hạn.

### **2.2. Yếu tố hỗ trợ (Catalysts)**

* Chiến lược mở rộng đội tàu lên quy mô lớn hơn.
* Tăng trưởng doanh thu vận tải và thương mại ổn định.
* Giá thuê tàu định hạn duy trì mặt bằng cao.

### **2.3. Rủi ro (Risks)**

* Chi phí đầu tư và giá tàu cao hơn dự kiến.
* Chi phí mua ngoài tăng khi mở rộng tuyến vận tải quốc tế.
* Biến động địa chính trị và thương mại toàn cầu làm giảm nhu cầu vận tải dầu.

---

## **3. Các trụ cột đầu tư (Pillars)**

### **3.1. Mở rộng đội tàu – Động lực tăng trưởng dài hạn**

* **Chiến lược:** PVT đặt mục tiêu sở hữu ~100 tàu vào năm 2030, tập trung vào tàu dầu thành phẩm và hóa chất – các phân khúc có biên lợi nhuận cao và ít cạnh tranh.
* **Hiện trạng:** Cuối 2024, PVT có **58 tàu**; dự kiến ~80 tàu vào 2030 do giá tàu cao khiến tiến độ đầu tư chậm.
* **Ý nghĩa:** Quy mô đội tàu lớn giúp tăng doanh thu vận tải và bù đắp xu hướng giảm giá cước.

### **3.2. Giá cước vận tải dầu thô chịu áp lực nhưng ổn định trở lại từ 2026**

* Giá cước ngắn hạn chịu ảnh hưởng từ dư cung tàu và nhu cầu dầu thô giảm.
* Tuy nhiên, dự báo sẽ ổn định trong giai đoạn 2026–2027 nhờ nhu cầu lọc hóa dầu nội địa và khu vực.
* **Dự báo giá cước:** 40.887 – 41.296 – 41.502 USD/ngày giai đoạn 2025–2027.

### **3.3. Tăng trưởng doanh thu ổn định nhờ đa dạng hóa kinh doanh**

* PVT đang mở rộng vận tải hóa chất và hàng rời – phân khúc ít cạnh tranh và biên lợi nhuận tốt hơn.
* **Dự báo doanh thu thuần:**

  * 2024: 14.687 tỷ đồng
  * 2025–2026: tăng trưởng CAGR ~8,4%, lên **16.268 tỷ đồng** năm 2026.
* Động lực doanh thu đến từ thương mại hóa chất và mở rộng tuyến vận tải quốc tế.

### **3.4. Chi phí tài chính và chi phí mua ngoài tăng – Áp lực trong ngắn hạn**

* Chi phí tài chính Q3/2025 tăng **37,4%** do lỗ tỷ giá.
* Biên lợi nhuận gộp giảm từ **16,9% (2025E)** so với **16,1% (2024)**.
* Chi phí dịch vụ mua ngoài tăng khi mở rộng đội tàu và khai thác tuyến quốc tế.
* Xu hướng chi phí có thể cải thiện từ 2026 khi đầu tư mới vào khai thác và tối ưu vận hành.

---

## **4. Các điểm nhấn tích cực & tiêu cực**

### **4.1. Điểm tích cực (Positives)**

* Doanh nghiệp dẫn đầu vận tải dầu khí với nền tảng tài chính mạnh.
* Doanh thu 9T2025 tăng **36,2% yoy** nhờ mở rộng thương mại và vận tải.
* Mở rộng đội tàu giúp củng cố tăng trưởng doanh thu và lợi nhuận dài hạn.
* Giá thuê tàu định hạn duy trì ổn định, định giá EV/EBITDA thấp hơn mức trung bình 5 năm.

### **4.2. Điểm tiêu cực (Negatives)**

* Lợi nhuận Q3/2025 giảm **33,1%** do biên lợi nhuận thu hẹp.
* Tiến độ đầu tư chậm do giá tàu và chi phí capital expenditure cao.
* Chi phí dịch vụ mua ngoài tăng cao gây áp lực lên biên lợi nhuận.
* Giá cước vận tải dầu thô giảm trong ngắn hạn do dư cung tàu.

### **4.3. Danh mục theo dõi (Watchlist)**

* Cập nhật diễn biến giá cước vận tải tàu dầu, tàu hóa chất, tàu hàng rời.
* Tiến độ mua mới tàu và chi phí đầu tư so với kế hoạch.
* Ảnh hưởng từ chiến sự, các lệnh trừng phạt hoặc gián đoạn logistics toàn cầu.
* Biến động tỷ giá ảnh hưởng chi phí vay và đánh giá lại nợ.

---

## **5. Kết luận**

PVT tiếp tục khẳng định vị thế dẫn đầu trong ngành vận tải dầu khí, với hướng đi dài hạn là mở rộng đội tàu và đa dạng hóa dịch vụ vận tải. Dù 2025 đối mặt với áp lực lợi nhuận từ biến động tỷ giá và chi phí cao, triển vọng 2026–2030 vẫn tích cực nhờ tăng trưởng doanh thu ổn định.

Với mức tăng giá kỳ vọng **18.6%**, khuyến nghị **KHẢ QUAN** được đưa ra cho cổ phiếu PVT.






# 🔎 **BÁO CÁO PHÂN TÍCH – PVT (Tổng CTCP Vận tải Dầu khí)**

**Khuyến nghị: KHẢ QUAN • Giá mục tiêu 22.000 VNĐ/cp • Upside: 18.6%**

---

## **1. Kết luận đầu tư (Investment Thesis)**

PVT là doanh nghiệp vận tải dầu khí hàng đầu Việt Nam với nền tảng tài chính vững và đội tàu lớn nhất thị trường nội địa. Doanh thu 9T2025 tăng trưởng tích cực nhờ mở rộng mảng vận tải và thương mại, song lợi nhuận Q3/2025 giảm do biên lợi nhuận thu hẹp và chi phí tài chính tăng.

Trong trung – dài hạn, động lực tăng trưởng đến từ chiến lược mở rộng đội tàu lên 80–100 chiếc và đa dạng hóa mảng kinh doanh sang hóa chất, hàng rời và tàu thành phẩm. Giá cho thuê tàu định hạn duy trì ổn định giúp tạo mặt bằng doanh thu bền vững, trong khi chi phí có thể giảm từ 2026 nhờ tối ưu vận hành.

Với tiềm năng tăng giá 18.6%, báo cáo đưa ra **khuyến nghị KHẢ QUAN**.

---

## **2. Khuyến nghị và giá mục tiêu**

### **2.1. Khuyến nghị KHẢ QUAN**

* **Giá mục tiêu:** 22.000 VNĐ/cp
* **Upside 12 tháng:** **18.6%**
* **Đánh giá:** Triển vọng tăng trưởng dài hạn hấp dẫn nhờ mở rộng đội tàu và đa dạng hóa hoạt động kinh doanh, dù chịu áp lực lợi nhuận trong ngắn hạn.

### **2.2. Yếu tố hỗ trợ (Catalysts)**

* Chiến lược mở rộng đội tàu lên quy mô lớn hơn.
* Tăng trưởng doanh thu vận tải và thương mại ổn định.
* Giá thuê tàu định hạn duy trì mặt bằng cao.

### **2.3. Rủi ro (Risks)**

* Chi phí đầu tư và giá tàu cao hơn dự kiến.
* Chi phí mua ngoài tăng khi mở rộng tuyến vận tải quốc tế.
* Biến động địa chính trị và thương mại toàn cầu làm giảm nhu cầu vận tải dầu.

---

## **3. Các trụ cột đầu tư (Pillars)**

### **3.1. Mở rộng đội tàu – Động lực tăng trưởng dài hạn**

* **Chiến lược:** PVT đặt mục tiêu sở hữu ~100 tàu vào năm 2030, tập trung vào tàu dầu thành phẩm và hóa chất – các phân khúc có biên lợi nhuận cao và ít cạnh tranh.
* **Hiện trạng:** Cuối 2024, PVT có **58 tàu**; dự kiến ~80 tàu vào 2030 do giá tàu cao khiến tiến độ đầu tư chậm.
* **Ý nghĩa:** Quy mô đội tàu lớn giúp tăng doanh thu vận tải và bù đắp xu hướng giảm giá cước.

### **3.2. Giá cước vận tải dầu thô chịu áp lực nhưng ổn định trở lại từ 2026**

* Giá cước ngắn hạn chịu ảnh hưởng từ dư cung tàu và nhu cầu dầu thô giảm.
* Tuy nhiên, dự báo sẽ ổn định trong giai đoạn 2026–2027 nhờ nhu cầu lọc hóa dầu nội địa và khu vực.
* **Dự báo giá cước:** 40.887 – 41.296 – 41.502 USD/ngày giai đoạn 2025–2027.

### **3.3. Tăng trưởng doanh thu ổn định nhờ đa dạng hóa kinh doanh**

* PVT đang mở rộng vận tải hóa chất và hàng rời – phân khúc ít cạnh tranh và biên lợi nhuận tốt hơn.
* **Dự báo doanh thu thuần:**

  * 2024: 14.687 tỷ đồng
  * 2025–2026: tăng trưởng CAGR ~8,4%, lên **16.268 tỷ đồng** năm 2026.
* Động lực doanh thu đến từ thương mại hóa chất và mở rộng tuyến vận tải quốc tế.

### **3.4. Chi phí tài chính và chi phí mua ngoài tăng – Áp lực trong ngắn hạn**

* Chi phí tài chính Q3/2025 tăng **37,4%** do lỗ tỷ giá.
* Biên lợi nhuận gộp giảm từ **16,9% (2025E)** so với **16,1% (2024)**.
* Chi phí dịch vụ mua ngoài tăng khi mở rộng đội tàu và khai thác tuyến quốc tế.
* Xu hướng chi phí có thể cải thiện từ 2026 khi đầu tư mới vào khai thác và tối ưu vận hành.

---

## **4. Các điểm nhấn tích cực & tiêu cực**

### **4.1. Điểm tích cực (Positives)**

* Doanh nghiệp dẫn đầu vận tải dầu khí với nền tảng tài chính mạnh.
* Doanh thu 9T2025 tăng **36,2% yoy** nhờ mở rộng thương mại và vận tải.
* Mở rộng đội tàu giúp củng cố tăng trưởng doanh thu và lợi nhuận dài hạn.
* Giá thuê tàu định hạn duy trì ổn định, định giá EV/EBITDA thấp hơn mức trung bình 5 năm.

### **4.2. Điểm tiêu cực (Negatives)**

* Lợi nhuận Q3/2025 giảm **33,1%** do biên lợi nhuận thu hẹp.
* Tiến độ đầu tư chậm do giá tàu và chi phí capital expenditure cao.
* Chi phí dịch vụ mua ngoài tăng cao gây áp lực lên biên lợi nhuận.
* Giá cước vận tải dầu thô giảm trong ngắn hạn do dư cung tàu.

### **4.3. Danh mục theo dõi (Watchlist)**

* Cập nhật diễn biến giá cước vận tải tàu dầu, tàu hóa chất, tàu hàng rời.
* Tiến độ mua mới tàu và chi phí đầu tư so với kế hoạch.
* Ảnh hưởng từ chiến sự, các lệnh trừng phạt hoặc gián đoạn logistics toàn cầu.
* Biến động tỷ giá ảnh hưởng chi phí vay và đánh giá lại nợ.

---

## **5. Kết luận**

PVT tiếp tục khẳng định vị thế dẫn đầu trong ngành vận tải dầu khí, với hướng đi dài hạn là mở rộng đội tàu và đa dạng hóa dịch vụ vận tải. Dù 2025 đối mặt với áp lực lợi nhuận từ biến động tỷ giá và chi phí cao, triển vọng 2026–2030 vẫn tích cực nhờ tăng trưởng doanh thu ổn định.

Với mức tăng giá kỳ vọng **18.6%**, khuyến nghị **KHẢ QUAN** được đưa ra cho cổ phiếu PVT.




In [ ]:
Phiên ngày mười tháng mười một vi en in đét tiếp tục phiên giảm với mức giảm một chấm mười sáu phần trăm tương ứng với mười tám chấm năm mươi sáu điểm
Thanh khoản vi en in đét chỉ đạt mức hơn hai mươi mốt nghìn tỷ đây là mức thanh khoản yếu. Khối ngoại tiếp tục duy trì đà bán ròng
Tính đến 14:30, dòng tiền ròng chủ động tại vi en ba mươi đang âm chín mươi sáu chấm bảy mươi hai tỷ đồng. Dòng tiền rút khỏi midcap với giá trị một nghìn một trăm bảy mươi ba tỷ đồng
Dòng tiền ròng tiếp tục rút khỏi nhóm ngành ngân hàng, chứng khoán và bất động sản trong phiên hôm nay
Hôm nay thiếu vắng dòng tiền lớn của các sát mua và bán chủ động